In [ ]:
## linear regression exercise [ build ridge and lasso]

## target : likes 

In [1]:
file=r'/Users/lalitsachan/Dropbox/Trainings/DeloitteMarchPythonMLHydMarch2020/facebook_comments.csv'

In [2]:
import pandas as pd
fb=pd.read_csv(file)

In [3]:
fb.head()

,likes,checkins,talking_about,page_category,feat5,feat6,feat7,feat8,feat9,feat10,...,CC4,CC5,Base_Time,Post_Length,Post Share Count,Post Promotion Status,H local,Comments_in_next_H_hrs,Post Published Weekday,Base Date Time Weekday
0,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,65,166,2,0,24,0,Wednesday,Saturday
1,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,10,132,1,0,24,0,Thursday,Friday
2,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,14,133,2,0,24,0,Friday,Saturday
3,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,7,-3,62,131,1,0,24,0,Friday,Monday
4,634995,0,463,Product/service,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,58,142,5,0,24,0,Monday,Wednesday


In [9]:
fb.shape

(40949, 42)

In [10]:
###
# replace sunday,monday... with numbers ‘Post Published Weekday’and’Base Date Time Weekday’
# covert them to their cyclic represenation 
# create dummies for page_category with frequency cutoff 500

# break your data in to 80/20
# build ridge and lasso models [ with grid search for penalty] 
# regression models on 80% of the data and check their performance(mae) on 20%
day_num_dict={'Sunday':1,'Monday':2,'Tuesday':3,
              'Wednesday':4,'Thursday':5,'Friday':6,'Saturday':7}

In [11]:
import numpy as np

In [12]:
for col in ['Post Published Weekday','Base Date Time Weekday']:
    fb[col].replace(day_num_dict,inplace=True)
    name_sin=col+'_sin'
    name_cos=col+'_cos'
    fb[name_sin]=np.sin(2*np.pi*fb[col]/7)
    fb[name_cos]=np.cos(2*np.pi*fb[col]/7)
    
    del fb[col]

In [15]:
# fb.columns

In [16]:
col='page_category'
freqs=fb[col].value_counts()
    
k=freqs.index[freqs>500][:-1]

for cat in k:
    name=col+'_'+cat
    fb[name]=(fb[col]==cat).astype(int)

del fb[col]


In [17]:
fb.shape

(40949, 57)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
fb_train,fb_test=train_test_split(fb,test_size=0.2,random_state=2)

In [20]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [21]:
lm=Ridge(fit_intercept=True)

In [29]:
params={'alpha':np.linspace(1,10000,100)}

In [30]:
grid=GridSearchCV(lm,param_grid=params,cv=10,scoring='neg_mean_absolute_error',
                 n_jobs=-1,verbose=20)

In [31]:
x_train=fb_train.drop(['likes'],1)
y_train=fb_train['likes']

In [33]:
grid.fit(x_train,y_train)

In [34]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [35]:
report(grid.cv_results_,5)

Model with rank: 1
Mean validation score: -1410279.57688 (std: 91866.79822)
Parameters: {'alpha': 2930.0}

Model with rank: 2
Mean validation score: -1410279.80935 (std: 91884.29938)
Parameters: {'alpha': 2829.0}

Model with rank: 3
Mean validation score: -1410280.97152 (std: 91851.72259)
Parameters: {'alpha': 3031.0}

Model with rank: 4
Mean validation score: -1410285.44647 (std: 91904.82007)
Parameters: {'alpha': 2728.0}

Model with rank: 5
Mean validation score: -1410285.53149 (std: 91838.67087)
Parameters: {'alpha': 3132.0}



In [37]:
test_pred=grid.predict(fb_test.drop(['likes'],1))

In [38]:
from sklearn.metrics import mean_absolute_error

In [39]:
mean_absolute_error(test_pred,fb_test['likes'])

1345426.5151445386

In [ ]:
## logistic regression

In [4]:
file=r'/Users/lalitsachan/Dropbox/Trainings/DeloitteMarchPythonMLHydMarch2020/default of credit card clients.xls'

In [7]:
cc=pd.read_excel(file,skiprows=1)

In [8]:
cc.head()
# target is default payment next month

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [40]:
# create dummies for SEX,EDUCATION,MARRIAGE
# drop ID
# Target  : default payment next month
# split data into 80,20
# build a logistic regression model [ tune for C, penalty and class_weight ]
# find a cutoff on the basis of f1 score
for col in ['SEX','EDUCATION','MARRIAGE']:
    dummy=pd.get_dummies(cc[col],prefix=col,drop_first=True)
    cc=pd.concat([cc,dummy],1)
    
    del cc[col]
del dummy

In [41]:
x=cc.drop(['default payment next month','ID'],1)
y=cc['default payment next month']

In [42]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [43]:
x_train.shape

(24000, 30)

In [44]:
y_train.shape

(24000,)

In [55]:
params={'C':np.linspace(1,1000,5),
       'class_weight':['balanced',None],
       'penalty':['l1','l2']}

In [56]:
from sklearn.linear_model import LogisticRegression

In [57]:
logr=LogisticRegression(fit_intercept=True)

In [58]:
grid=GridSearchCV(logr,param_grid=params,cv=5,scoring='roc_auc',n_jobs=-1,
                 verbose=20)

In [60]:
grid.fit(x_train,y_train)

In [64]:
report(grid.cv_results_,5)

In [65]:
train_score=grid.predict_proba(x_train)[:,1]

In [66]:
real=y_train

In [67]:
cutoffs=np.linspace(0.01,0.99,99)
F1_all=[]


for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
    
      
    precision=TP/(TP+FP)
    recall=TP/P
    
    F1=(2*precision*recall)/(precision+recall)
    
    F1_all.append(F1)
    

In [68]:
mycutoff=cutoffs[F1_all==max(F1_all)][0]
mycutoff

0.59